In [1]:
import pandas as pd
import numpy as np

In [ ]:
pixel_to_micron = 2.43 # 10X power

In [66]:
img_size = pd.read_csv('../data/power.txt', sep=r'\t')

D:\Miniconda3\envs\sci\lib\site-packages\ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [67]:
img_size['Name'] = img_size['Name'].str.split('.').str[0]

In [68]:
img_size['Size'] = img_size['Size'].str.split(',')

In [15]:
img_sel = pd.read_csv('../data/summary.txt', sep=r'\t', header=None)

D:\Miniconda3\envs\sci\lib\site-packages\ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [18]:
img_sel.columns = ['name', 'num']

In [96]:
img_comp = img_sel.merge(img_size, left_on='name', right_on='Name').drop('Name', axis=1)

In [97]:
len(img_comp)

799

In [99]:
img_comp['area'] = img_comp['Size'].apply(lambda x:int(x[0].replace("'", "")) * int(x[1].replace("'", "")))

In [100]:
tile_area = 96 * 96
img_comp['ratio'] = (img_comp['Power'] / 10) ** 2 * tile_area * img_comp['num']  / img_comp['area']

In [101]:
img_comp['ori_num_10X'] = img_comp['area'] / ((img_comp['Power'] / 10) ** 2 * tile_area)

In [102]:
img_comp.loc[:,['num', 'ori_num_10X']].sum()

num            2.451661e+06
ori_num_10X    4.609560e+07
dtype: float64

In [88]:
2.451661e+06 / 4.609560e+07

0.053186442957679256

In [103]:
img_comp.loc[img_comp['num'] == img_comp['num'].max()]

,name,num,Power,Size,area,ratio,ori_num_10X
479,TCGA-D8-A1XR-01Z-00-DX1,27971,40,"['94468', '184688']",17447105984,0.2364,118320.759983


In [104]:
img_comp.loc[img_comp['ratio'] == img_comp['ratio'].max()]

,name,num,Power,Size,area,ratio,ori_num_10X
459,TCGA-D8-A1X5-01Z-00-DX2,26666,40,"['81748', '101591']",8304861068,0.473465,56320.943658


In [107]:
special = img_comp.loc[img_comp['num'] <= 50, 'name']

In [109]:
len(special)

55

In [111]:
target = pd.read_excel('../data/Target.xlsx')

In [115]:
special_df = target.set_index('sample').loc[[case[:12] for case in special]]

In [120]:
special_df.sort_values('duration', inplace=True)

In [123]:
# special.to_csv('../data/less50.txt', header=False)
img_comp.to_csv('../data/img_comp.xlsx')

In [94]:
img_comp['ratio'].nsmallest(10)

732    0.000010
723    0.000015
706    0.000016
721    0.000020
745    0.000022
598    0.000029
705    0.000043
711    0.000043
730    0.000045
597    0.000048
Name: ratio, dtype: float64

In [93]:
img_comp['ratio'].nlargest(10)

459    0.473465
458    0.390258
389    0.375826
460    0.326863
449    0.297690
691    0.286293
87     0.283159
46     0.278065
170    0.253598
164    0.248040
Name: ratio, dtype: float64

In [77]:
img_comp.describe()

,num,Power,area,ratio
count,799.000000,799.000000,7.990000e+02,799.000000
mean,3068.411765,38.448060,7.673671e+09,0.059068
std,3491.340687,5.354079,3.521220e+09,0.061477
min,1.000000,20.000000,3.693566e+08,0.000010
25%,550.500000,40.000000,5.381896e+09,0.012189
50%,1971.000000,40.000000,7.747655e+09,0.041195
75%,4467.500000,40.000000,9.846987e+09,0.087751
max,27971.000000,40.000000,3.562163e+10,0.473465


### in real disk

In [1]:
import os

In [3]:
base = '../../selected/'
img_disk = {}
for dn in os.listdir(base):
    img_disk[dn] = len(os.listdir(os.path.join(base, dn)))

In [5]:
import pandas as pd

In [11]:
disk_df = pd.DataFrame.from_dict(img_disk, orient='index')

In [14]:
diff_df = disk_df.merge(img_sel, left_index=True, right_on='name')

In [20]:
diff_df['gap'] = abs(comp_df.iloc[:, 0] - comp_df.iloc[:, 2]) / comp_df.iloc[:, 0]

In [21]:
diff_df.loc[comp_df['gap'] > 0.1]

,0,case,num,gap
161,209,TCGA-AC-A3W5-01Z-00-DX1,182,0.129187


Why the imgs saved are less than the record in dict?

In [ ]:
diff = set(img_sel['name'].tolist()) - set(diff_df['case'].tolist())